In [85]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# data split
from sklearn.model_selection import train_test_split

# nlp preprocessing
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import contractions # to deal with english contractions

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from collections import Counter

from transformers import AutoTokenizer

[nltk_data] Downloading package stopwords to /Users/ygao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ygao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ygao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [146]:
EN_FILE_PATH = '../data/nl-en/europarl-v7.nl-en.en'
NL_FILE_PATH = '../data/nl-en/europarl-v7.nl-en.nl'
CLEAN_DATA_PATH = '../data/clean/'

RANDOM_SEED = 297
MAX_INPUT_LENGTH = 50

In [3]:
# load data into data frames
def load(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
        lines = [line.strip() for line in lines]
        data = pd.DataFrame({'Text': lines})
        return data
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error loading file '{file_path}': {e}")
        return None

In [4]:
# custom print function to check the dataframe
def printCheck(df):
    for index, row in df.head().iterrows():
        print(f"{index}: {row['Text'][:1000]}")

In [5]:
en_data = load(EN_FILE_PATH)
nl_data = load(NL_FILE_PATH)
print('English Corpus')
printCheck(en_data)
print('Dutch Corpus')
printCheck(nl_data)

English Corpus
0: Resumption of the session
1: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
2: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
3: You have requested a debate on this subject in the course of the next few days, during this part-session.
4: In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.
Dutch Corpus
0: Hervatting van de zitting
1: Ik verklaar de zitting van het Europees Parlement, die op vrijdag 17 december werd onderbroken, te zijn hervat. Ik wens u allen een gelukkig nieuwjaar en hoop d

In [6]:
# train, val, test split
# reorder and reassign the indeces
en_train, en_test, nl_train, nl_test = train_test_split(en_data, nl_data, test_size=0.1, random_state=RANDOM_SEED)
en_test.sort_index(inplace=True, ignore_index=True)
nl_test.sort_index(inplace=True, ignore_index=True)
en_train, en_val, nl_train, nl_val = train_test_split(en_train, nl_train, test_size=0.11, random_state=RANDOM_SEED)
en_train.sort_index(inplace=True, ignore_index=True)
nl_train.sort_index(inplace=True, ignore_index=True)
en_val.sort_index(inplace=True, ignore_index=True)
nl_val.sort_index(inplace=True, ignore_index=True)

In [7]:
printCheck(en_train)
printCheck(nl_train)

0: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
1: In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.
2: Please rise, then, for this minute' s silence.
3: (The House rose and observed a minute' s silence)
4: Madam President, on a point of order.
0: Ik verklaar de zitting van het Europees Parlement, die op vrijdag 17 december werd onderbroken, te zijn hervat. Ik wens u allen een gelukkig nieuwjaar en hoop dat u een goede vakantie heeft gehad.
1: Nu wil ik graag op verzoek van een aantal collega's een minuut stilte in acht nemen ter nagedachtenis van de slachtoffers. Ik doel hiermee met name op de slachtoffers van het noodweer dat verschillende lids

In [125]:
# clean text

def clean_text(df, eng = True):
    cleaned_texts = []
    lemmatizer = WordNetLemmatizer() #not used
    if eng: #not used either for now
        stop_words = stopwords.words('english')
    else:
        stop_words = stopwords.words('dutch')
        
    for index, row in df.iterrows():
        text = row['Text']
        text = text.lower() #covert to lower case
        text = re.sub(r'<.*?>', '', text) #remove html tag
        text = re.sub(r'http\S+', '', text) #remove url
        text = re.sub(r'\[.*?\]', '', text) #remove everything in between square brackets
        text = re.sub(r'\(.*?\)', '', text) #remove everything in between parentheses
        text = re.sub(r'\b[A-Z]\d-\d{4}/\d{4}\b', '', text) #deleting more weird stuff
        text = contractions.fix(text) #extend english contractions
        text = re.sub(r'[^A-Za-z0-9\s\'s]', '', text) #keep only alphabets, numbers, and space, maybe add in' as well for possessive
        
        cleaned_texts.append(text)
    #return the text in a dataframe
    df_cleaned = pd.DataFrame(cleaned_texts, columns=['Text'])
    return df_cleaned, cleaned_texts

In [126]:
# save df
def save_df(df, path):
    df_name = [name for name, val in globals().items() if val is df][0]
    file_name = f"{df_name}.csv"
    file_path = os.path.join(path, file_name)
    df.to_csv(file_path, index=False)

In [127]:
# clean text and check
df_en_train_clean, en_train_clean = clean_text(en_train)
df_nl_train_clean, nl_train_clean = clean_text(nl_train, eng=False)
df_en_val_clean, en_val_clean = clean_text(en_val)
df_nl_val_clean, nl_val_clean = clean_text(nl_val, eng=False)
df_en_test_clean, en_test_clean = clean_text(en_test)
df_nl_test_clean, nl_test_clean = clean_text(nl_test, eng=False)

In [128]:
save_df(df_en_train_clean, CLEAN_DATA_PATH)
save_df(df_en_val_clean, CLEAN_DATA_PATH)
save_df(df_en_test_clean, CLEAN_DATA_PATH)
save_df(df_nl_train_clean, CLEAN_DATA_PATH)
save_df(df_nl_val_clean, CLEAN_DATA_PATH)
save_df(df_nl_test_clean, CLEAN_DATA_PATH)

In [37]:
printCheck(df_en_train_clean)
printCheck(df_nl_train_clean)

0: i declare resumed the session of the european parliament adjourned on friday 17 december 1999 and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period
1: in the meantime i should like to observe a minute' s silence as a number of members have requested on behalf of all the victims concerned particularly those of the terrible storms in the various countries of the european union
2: please rise then for this minute' s silence
3: the house rose and observed a minute' s silence
4: madam president on a point of order
0: ik verklaar de zitting van het europees parlement die op vrijdag 17 december werd onderbroken te zijn hervat ik wens you allen een gelukkig nieuwjaar en hoop dat you een goede vakantie heeft gehad
1: nu wil ik graag op verzoek van een aantal collega's een minuut stilte in acht nemen ter nagedachtenis van de slachtoffers ik doel hiermee met name op de slachtoffers van het noodweer dat verschillende lidstaten van de uni

In [40]:
print(en_train_clean[:10])
print(nl_train_clean[:10])

['i declare resumed the session of the european parliament adjourned on friday 17 december 1999 and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period', "in the meantime i should like to observe a minute' s silence as a number of members have requested on behalf of all the victims concerned particularly those of the terrible storms in the various countries of the european union", "please rise then for this minute' s silence", "the house rose and observed a minute' s silence", 'madam president on a point of order', 'you will be aware from the press and television that there have been a number of bomb explosions and killings in sri lanka', 'one of the people assassinated very recently in sri lanka was mr kumar ponnambalam who had visited the european parliament just a few months ago', "would it be appropriate for you madam president to write a letter to the sri lankan president expressing parliament's regret at his and the other vi

In [129]:
# tokenize and add in special tokens
def tokenize_text(df):
    tk_text = []
    for index, row in df.iterrows():
        text = row['Text']
        tokens = word_tokenize(text)
        tokens = ['<bos>'] + tokens + ['<eos>']
        tk_text.append(tokens)
    return tk_text


In [130]:
en_train_tk = tokenize_text(df_en_train_clean)
nl_train_tk = tokenize_text(df_nl_train_clean)
en_val_tk = tokenize_text(df_en_val_clean)
nl_val_tk = tokenize_text(df_nl_val_clean)
en_test_tk = tokenize_text(df_en_test_clean)
nl_test_tk = tokenize_text(df_nl_test_clean)

In [132]:
max_sentence_length = max(len(sentence) for sequence in en_train_tk for sentence in sequence)
print(max_sentence_length)

41


In [121]:
print(en_train_tk[:10])
print(nl_train_tk[:10])

[['<bos>', 'i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period', '<eos>'], ['<bos>', 'in', 'the', 'meantime', 'i', 'should', 'like', 'to', 'observe', 'a', 'minute', "'", 's', 'silence', 'as', 'a', 'number', 'of', 'members', 'have', 'requested', 'on', 'behalf', 'of', 'all', 'the', 'victims', 'concerned', 'particularly', 'those', 'of', 'the', 'terrible', 'storms', 'in', 'the', 'various', 'countries', 'of', 'the', 'european', 'union', '<eos>'], ['<bos>', 'please', 'rise', 'then', 'for', 'this', 'minute', "'", 's', 'silence', '<eos>'], ['<bos>', 'the', 'house', 'rose', 'and', 'observed', 'a', 'minute', "'", 's', 'silence', '<eos>'], ['<bos>', 'madam', 'president', 'on', 'a', 'point', 'of', 'order', '<eos>'], ['<bos>', 'you', 'wi

In [122]:
# vocabulary
def word_index(text):
    wordcount = Counter()
    for sentence in text:
        wordcount.update(sentence)
    
    vocab = [word for word, count in wordcount.most_common()]
    word_to_index = {word: idx for idx, word in enumerate(vocab)}
    index_to_word = {idx: word for idx, word in enumerate(vocab)}
    
    return word_to_index, index_to_word

In [123]:
en_word2idx, en_idx2word = word_index(en_train_tk)
nl_word2idx, nl_idx2word = word_index(nl_train_tk)

In [131]:
print(en_word2idx)

{'the': 0, '<bos>': 1, '<eos>': 2, 'of': 3, 'to': 4, 'and': 5, 'in': 6, 'that': 7, 'a': 8, 'is': 9, 'for': 10, 'we': 11, 'i': 12, 'this': 13, 'on': 14, 'it': 15, 'be': 16, 'not': 17, 'are': 18, 'as': 19, 'have': 20, 'with': 21, 'which': 22, 'european': 23, 'will': 24, 'by': 25, 'has': 26, 'mr': 27, 'at': 28, 'commission': 29, 'an': 30, 'would': 31, 'also': 32, 'all': 33, 'but': 34, 'should': 35, 'can': 36, 'our': 37, 'from': 38, 'must': 39, 'president': 40, 'there': 41, 'been': 42, 'you': 43, 'union': 44, "'s": 45, 'parliament': 46, 'states': 47, 'member': 48, 'more': 49, 'was': 50, 'report': 51, 'its': 52, 'these': 53, 'or': 54, 'they': 55, 'do': 56, 'their': 57, 'council': 58, 'like': 59, 'what': 60, 'very': 61, 'if': 62, 'one': 63, 'europe': 64, 'so': 65, 'countries': 66, 'eu': 67, 'us': 68, 'my': 69, "'": 70, 'no': 71, 'about': 72, 'other': 73, 'people': 74, 'need': 75, 'because': 76, 'who': 77, 'only': 78, 'policy': 79, 'important': 80, 'new': 81, 'time': 82, 'now': 83, 'such': 84

In [139]:
# convert text to vec
def convert_text(text, word2idx):
    text_vec = []
    for sentence in text:
        vec = [word2idx.get(word, 0) for word in sentence] #unknown vocab -> 0
        text_vec.append(vec)
    return text_vec

In [140]:
# pad the sentences
def padding_sequence(sequence, max_len, padding_value = 0):
    padded = tf.keras.preprocessing.sequence.pad_sequences (
        sequence, maxlen = max_len, 
        padding = 'post', truncating='post',
        value = padding_value
    )
    return padded

In [144]:
en_train_seq = convert_text(en_train_tk, en_word2idx)
nl_train_seq = convert_text(nl_train_tk, nl_word2idx)
en_val_seq = convert_text(en_val_tk, en_word2idx)
nl_val_seq = convert_text(nl_val_tk, nl_word2idx)
en_test_seq = convert_text(en_test_tk, en_word2idx)
nl_test_seq = convert_text(nl_test_tk, nl_word2idx)

In [150]:
en_train_padded = padding_sequence(en_train_seq, MAX_INPUT_LENGTH)
nl_train_padded = padding_sequence(nl_train_seq, MAX_INPUT_LENGTH)
en_val_padded = padding_sequence(en_val_seq, MAX_INPUT_LENGTH)
nl_val_padded = padding_sequence(nl_val_seq, MAX_INPUT_LENGTH)
en_test_padded = padding_sequence(en_test_seq, MAX_INPUT_LENGTH)
nl_test_padded = padding_sequence(nl_test_seq, MAX_INPUT_LENGTH)